In [ ]:
import requests # for making API requests
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime
import json

# 1. Extract: Fetch data from CoinGecko API
url = "https://api.coingecko.com/api/v3/coins/markets"
params = {
    "vs_currency": "usd", # Prices will be returned in US dollars.
    "order": "market_cap_desc", # Sort coins by descending market cap (largest first).
    "per_page": 10,  # Ask API to return only 10 results per page.
    "page": 1, # Get the first page.
    "sparkline": False # this line excludes sparkline data. Sparkline data can be large and is not needed for this example.
}

response = requests.get(url, params=params)
data = response.json()
print(f"Raw JSON file data: {data}")  # Print raw data for debugging
print(f"lengths of data: {len(data)}")  # Print length of data for debugging


###############################################################
# Save to file
with open("coins_data.json", "w", encoding="utf-8") as f:
    json.dump(data, f, indent=4)  # indent=4 makes it pretty-printed

print("API response saved to coins_data.json")
###############################################################

# 2. Transform: Load into Pandas DataFrame
df = pd.DataFrame(data)

# Keep only relevant columns and rename them
df = df[["id", "symbol", "name", "current_price", "market_cap", "total_volume", "last_updated"]]
df.rename(columns={
    "id": "coin_id",
    "symbol": "coin_symbol",
    "name": "coin_name",
    "current_price": "price_usd",
    "market_cap": "market_cap_usd",
    "total_volume": "volume_usd",
    "last_updated": "last_updated"
}, inplace=True)

# Convert last_updated to datetime object
df["last_updated"] = pd.to_datetime(df["last_updated"])

# Optional: add extraction timestamp
df["extracted_at"] = datetime.now()

print(df.head())


# 3. Load: Save DataFrame to SQLite database
engine = create_engine("sqlite:///crypto_data.db") # This creates a new SQLite database file named crypto_data.db

# Create table if not exists, append data
df.to_sql("crypto_prices", con=engine, if_exists="append", index=False)

print("Data loaded into DB successfully!")


Raw JSON file data: [{'id': 'bitcoin', 'symbol': 'btc', 'name': 'Bitcoin', 'image': 'https://coin-images.coingecko.com/coins/images/1/large/bitcoin.png?1696501400', 'current_price': 117000, 'market_cap': 2330024171813, 'market_cap_rank': 1, 'fully_diluted_valuation': 2330024171813, 'total_volume': 36428096224, 'high_24h': 117888, 'low_24h': 116714, 'price_change_24h': -284.79029316939705, 'price_change_percentage_24h': -0.24282, 'market_cap_change_24h': -4472798295.894531, 'market_cap_change_percentage_24h': -0.1916, 'circulating_supply': 19922993.0, 'total_supply': 19922993.0, 'max_supply': 21000000.0, 'ath': 124128, 'ath_change_percentage': -5.78069, 'ath_date': '2025-08-14T00:37:02.582Z', 'atl': 67.81, 'atl_change_percentage': 172373.53982, 'atl_date': '2013-07-06T00:00:00.000Z', 'roi': None, 'last_updated': '2025-09-19T09:16:01.480Z'}, {'id': 'ethereum', 'symbol': 'eth', 'name': 'Ethereum', 'image': 'https://coin-images.coingecko.com/coins/images/279/large/ethereum.png?1696501628',